#  Comparativo de solver basado en multiplicadores de Lagrange y método de Newton

A continuación se muestra el comparativo entre los resultados obtenido con:

* a) el solver basado en los multiplicadores de Lagrange y 
* b) el solver con el método de Newton usando las aproximaciones con diferencias finitas y funciones simbólicas para representar las expresiones de derivadas que se involucran en su implementación. 


Cabe destacar que para su elaboración se consideraron las siguientes hipótesis: i) se decidió variar el valor del rendimiento $r$ en un rango de 0.4 a 1, ii) se muestra la distancia entre ambas soluciones de $w$ usando la norma 2 y la distancia usando la norma 1, iii) además se acompaña del retorno bajo ambos métodos de solución del Modelo de Markowitz, mostrando que cumple las restricciones lineales asociadas y iv) la varianza correspondiente para las soluciones obtenidas con dichas metodologías.



## Librerías

In [ ]:
import numpy as np
import pandas as pd
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt

In [ ]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [3]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [4]:
datos.head()

,005930.KS,0688.HK,0700.HK,1398.HK,2010.SR,7203.T,AAPL,AI.PA,AMT,AMZN,BABA,BHP.AX,BRK-A,CCI,COP,CVX,DLR,FB,GOOG,HD,HON,JNJ,JPM,KO,LIN,LMT,MA,MC.PA,MRK,MSFT,NSRGY,PEP,PFE,PG,PLD,RDSA.AS,RELIANCE.NS,RHHBY,RIO.L,RTX,T,UNH,UNP,UPS,V,VOO,VTI,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26600.0,24.704800,112.800003,5.77,79.50,7507.0,109.330002,89.786400,99.669998,308.519989,103.599998,27.603399,223600.0,79.510002,68.919998,112.580002,66.410004,78.449997,523.373108,103.430000,95.556229,104.519997,62.490002,42.139999,129.949997,193.309998,85.680000,130.850006,57.189999,46.759998,72.650002,94.440002,31.33,90.440002,43.430000,27.750000,442.774994,33.910000,2970.0,72.397736,33.869999,100.779999,118.610001,110.379997,66.254997,188.399994,105.919998,46.959999,85.900002,92.830002
2015-01-05,26660.0,24.951799,113.500000,5.80,79.50,7507.0,106.250000,87.005997,98.230003,302.190002,101.000000,27.547300,220980.0,79.000000,65.639999,108.080002,67.690002,77.190002,512.463013,101.260002,93.735291,103.790001,60.549999,42.139999,126.519997,189.289993,83.269997,127.050003,58.040001,46.330002,70.959999,93.730003,31.16,90.010002,43.400002,26.615000,437.924988,34.029999,2883.5,71.189430,33.549999,99.120003,114.599998,108.169998,64.792503,185.089996,104.099998,46.570000,85.650002,90.290001
2015-01-06,25900.0,24.605900,120.000000,5.71,77.00,7300.0,106.260002,86.279999,97.970001,295.290009,103.320000,26.267099,220450.0,78.849998,62.930000,108.029999,67.480003,76.150002,500.585632,100.949997,93.516014,103.279999,58.980000,42.459999,124.900002,188.399994,83.089996,125.599998,60.320000,45.650002,70.610001,93.019997,31.42,89.599998,43.549999,26.514999,418.049988,33.900002,2944.5,70.182503,33.599998,98.919998,112.230003,107.459999,64.375000,183.270004,103.080002,47.040001,86.309998,89.809998
2015-01-07,26140.0,24.507099,124.400002,5.75,78.25,7407.0,107.750000,86.669601,99.000000,298.420013,102.129997,26.267099,223480.0,80.500000,63.349998,107.940002,68.019997,76.150002,499.727997,104.410004,94.192909,105.559998,59.070000,42.990002,126.300003,190.830002,84.220001,125.699997,61.610001,46.230000,70.750000,95.739998,31.85,90.070000,44.209999,26.870001,427.149994,33.990002,2962.5,70.943993,33.169998,99.930000,112.849998,108.459999,65.237503,185.559998,104.309998,46.189999,88.599998,90.720001
2015-01-08,26280.0,23.864799,127.300003,5.72,79.25,7554.0,111.889999,90.317703,99.919998,300.459991,105.029999,26.519400,226680.0,81.769997,64.930000,110.410004,68.910004,78.180000,501.303680,106.720001,95.908974,106.389999,60.389999,43.509998,128.380005,195.130005,85.529999,129.649994,62.849998,47.590000,71.459999,97.480003,32.50,91.099998,44.220001,27.495001,421.024994,34.279999,3027.5,72.152298,33.500000,104.699997,117.080002,110.410004,66.112503,188.820007,106.150002,47.180000,90.470001,92.230003


In [ ]:
mu = aux.calcular_rendimiento(datos)

In [ ]:
S = aux.calcular_varianza(datos)

In [7]:
max(mu)

array(0.40221088)

In [ ]:
rango =np.arange(0.4,1.1,0.1)

In [9]:
rango

array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

### Comparativo vs Newton con Aproximación al Gradiente

In [ ]:
res_sol1 = [mkv.markowitz(r,mu,S) for r in rango]

In [ ]:
fo = lambda w: w@S@w
#w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#b = cp.array([r,1])
M = cp.ones((2,mu.shape[0]))
tol=1e-8
tol_backtracking=1e-14
#p_ast=fo(w_ast)

In [12]:
res_sol2 = [opt.Newtons_method_feasible_init_point(fo,
                                                   A,
                                                   utils.feasible_markowitz(r,mu),
                                                   tol,
                                                   tol_backtracking,
                                                   mkv.markowitz(r,mu,S),
                                                   fo(mkv.markowitz(r,mu,S)),
                                                   maxiter=50)[0] for r in rango]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.023	0.122	4.9545	658.4761	---		2466.2601
1	0.023	0.0097	10.457	68.1379	1	2466.2601
2	0.023	-0.0001	5.8945	7.9521	1	2466.2601
Error of x with respect to x_ast: 5.894541878242724
Approximate solution: [ 0.13771998 -0.09796931  0.43691112 -0.25868511 -0.08150009 -0.09940936
 -0.4716699  -0.13128719  0.11588815  0.33700828 -0.41721869  0.08081464
 -0.10128326  0.2209826  -0.31610939  0.07953399  0.10089517 -0.34534003
 -0.1673634  -0.27195041 -0.27361185 -0.07275707 -0.8088519  -0.51117351
 -0.15207089  0.26163102  0.61354895  0.08867941 -0.16949171 -0.73313469
  0.07103403 -0.01744836 -0.56446213 -0.3781923  -0.53554974 -0.24698683
  0.22894122 -0.1605263   0.04831162 -0.66392354 -0.792152   -0.14675077
 -0.30915918 -0.50678346 -0.90171979 -5.16667787 14.13189111  0.78862504
 -0.09158553 -0.77962076]
I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0243	0.1415	4.626	531.3532	---		5647.0399
1

La norma 2 está dada por

In [13]:
norm2 = np.zeros(7)
for i in range(7):
  norm2[i] = np.linalg.norm(res_sol1[i]-res_sol2[i], ord=2)
  print(np.linalg.norm(res_sol1[i]-res_sol2[i]))

14.48901677729033
0.018030787868378962
0.01035497008040384
0.10002078659529018
0.017146307606563396
55.95978724879813
15.136510396005391


La norma 1 está dada por

In [14]:
norm1 = np.zeros(7)
for i in range(7):
  norm1[i] = np.linalg.norm(res_sol1[i]-res_sol2[i], ord=1)
  print(np.linalg.norm(res_sol1[i]-res_sol2[i], ord=1))

31.763664646723473
0.03637170277350131
0.016891033326778493
0.1629955581973912
0.0334760000799008
142.19685945281984
24.167110272296522


La restricción del rendimiento del primer solver es:

In [15]:
mu1 = np.zeros(7)
for i in range(7):
  mu1[i] = sum(res_sol1[i]*mu)
  print(sum(res_sol1[i]*mu))

0.4000000000000004
0.5000000000000002
0.6000000000000004
0.7000000000000002
0.8
0.9000000000000002
1.0000000000000002


La restricción de rendimiento para el segundo solver es:

In [16]:
mu2 = np.zeros(7)
for i in range(7):
  mu2[i] = sum(res_sol2[i]*mu)
  print(sum(res_sol2[i]*mu))

0.399999999999998
0.49999999999999917
0.5999999999999979
0.7000000000000002
0.7999999999999988
0.899999999999997
0.9999999999999951


La restricción de unicidad para el primer solver es:

In [17]:
u1 = np.zeros(7)
for i in range(7):
  u1[i] = sum(res_sol1[i])
  print(sum(res_sol1[i]))

0.9999999999999999
0.9999999999999997
0.9999999999999997
1.0000000000000007
0.9999999999999993
0.9999999999999998
1.0000000000000018


La restricción de unicidad para el segundo solver es:

In [18]:

u2 = np.zeros(7)
for i in range(7):
  u2[i] = sum(res_sol2[i])
  print(sum(res_sol2[i]))

1.0
1.0000000000000062
1.0000000000000013
0.9999999999999991
0.999999999999998
0.9999999999999614
0.9999999999999875


La solución a la función objetivo para el primer solver es:

In [19]:
s1 = np.zeros(7)
for i in range(7):
  s1[i] = res_sol1[i]@S@res_sol1[i]
  print(res_sol1[i]@S@res_sol1[i])

9.397644610398304e-05
0.00012574783889287994
0.00016502479794085683
0.0002118073232479138
0.00026609541481405164
0.0003278890726392688
0.0003971882967235673


La solución a la funcióón objetivo para el segundo solver es:

In [20]:
s2 = np.zeros(7)
for i in range(7):
  s2[i] = res_sol2[i]@S@res_sol2[i]
  print(res_sol2[i]@S@res_sol2[i])

0.0008412905142421655
0.00012574827275669005
0.0001650248738326796
0.00021181319675961214
0.0002660957832459081
0.012665356258319905
0.0005269785878172568


In [ ]:
tabla = pd.DataFrame({'||Dif||': norm2,
                     '|Dif|': norm1,
                     'w1*mu': mu1,
                     'w2*mu': mu2,
                     'Sigma1': s1,
                     'Sigma2': s2})

**Soluciones de método de Lagrange ($L$) vs Newton usando diferencias finitas ($N_{df}$)**

In [22]:
tabla

,||Dif||,|Dif|,w1*mu,w2*mu,Sigma1,Sigma2
0,14.489017,31.763665,0.4,0.4,0.000094,0.000841
1,0.018031,0.036372,0.5,0.5,0.000126,0.000126
2,0.010355,0.016891,0.6,0.6,0.000165,0.000165
3,0.100021,0.162996,0.7,0.7,0.000212,0.000212
4,0.017146,0.033476,0.8,0.8,0.000266,0.000266
5,55.959787,142.196859,0.9,0.9,0.000328,0.012665
6,15.136510,24.167110,1.0,1.0,0.000397,0.000527


In [23]:
mu2

array([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])



De lo anterior, se desprenden los siguientes hallazgos:

* La diferencia entre las soluciones aumenta conforme aumenta el rendimiento deseado.

* A pesar del aumento en la diferencia, ambas soluciones cumplen con las restricciones deseadas.

* La solución es mejor usando el solver basado en los multiplicadores de Lagrange. 

### Comparativo vs Newton con funciones simbólicas

In [24]:
res_sol3 = [opt.Newtons_method_feasible_init_point(fo,
                                                   A,
                                                   utils.feasible_markowitz(r,mu),
                                                   tol,
                                                   tol_backtracking,
                                                   mkv.markowitz(r,mu,S),
                                                   fo(mkv.markowitz(r,mu,S)),
                                                   50,
                                                   opt.gfo_cp_mark,
                                                   opt.Hfo_cp_mark,
                                                   S)[0] for r in rango]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0115	0.0619	4.9545	658.4761	---		10080.7486
1	0.0115	-0.0	0.0	0.0	1	10080.7486
Error of x with respect to x_ast: 5.740873396029681e-12
Approximate solution: [ 1.58072889e-01 -2.07065137e-02  1.57018663e-01 -9.18400947e-02
  3.19613490e-02  5.13368560e-02  8.66470099e-02  1.30875327e-02
  3.68048577e-02  2.84552851e-01 -5.90047808e-03  2.19086962e-03
  2.02831720e-01  9.44790541e-02  2.24450410e-02  1.48916018e-02
  7.58576276e-03  2.97083297e-02  5.75104750e-02  1.97122290e-01
  1.19107538e-01  1.27459173e-01  1.39924071e-01  1.41741738e-02
  8.80157677e-02  1.49727253e-01  1.68115707e-01  7.64598163e-02
  8.04449497e-02  8.16111393e-02  1.91829256e-01 -2.43470176e-02
  2.65366644e-02  7.68672990e-02  2.15800786e-02 -1.07693804e-01
  1.63426962e-01  1.83045029e-02  6.13681468e-02 -1.07614337e-01
 -5.01292415e-02  1.37878242e-01  1.03089497e-01 -4.23273918e-02
  1.06201816e-02 -2.32473614e+00  1.92609722e-01  2.62

La norma 2 está dada por

In [26]:
norm2 = np.zeros(7)
for i in range(7):
  norm2[i] = np.linalg.norm(res_sol1[i]-res_sol3[i], ord=2)
  print(np.linalg.norm(res_sol1[i]-res_sol3[i]))

1.411129357794489e-11
1.6220153054390496e-11
1.5785109886227678e-11
2.4589903776705652e-11
2.3223462466169764e-11
2.23753157471894e-11
2.7473502099167238e-11


La norma 1 está dada por

In [27]:
norm1 = np.zeros(7)
for i in range(7):
  norm1[i] = np.linalg.norm(res_sol1[i]-res_sol3[i], ord=1)
  print(np.linalg.norm(res_sol1[i]-res_sol3[i], ord=1))

6.573455842151521e-11
7.96733798727467e-11
6.933233327732768e-11
1.0330760639304382e-10
9.366458997295268e-11
9.864391595226785e-11
1.199407483054582e-10


La restricción del rendimiento del primer solver es:

In [28]:
mu1 = np.zeros(7)
for i in range(7):
  mu1[i] = sum(res_sol1[i]*mu)
  print(sum(res_sol1[i]*mu))

0.4000000000000004
0.5000000000000002
0.6000000000000004
0.7000000000000002
0.8
0.9000000000000002
1.0000000000000002


La restricción de rendimiento para el segundo solver es:

In [29]:
mu2 = np.zeros(7)
for i in range(7):
  mu2[i] = sum(res_sol3[i]*mu)
  print(sum(res_sol3[i]*mu))

0.39999999999999863
0.4999999999999982
0.5999999999999976
0.6999999999999997
0.7999999999999975
0.8999999999999965
0.9999999999999971


La restricción de unicidad para el primer solver es:

In [30]:
u1 = np.zeros(7)
for i in range(7):
  u1[i] = sum(res_sol1[i])
  print(sum(res_sol1[i]))

0.9999999999999999
0.9999999999999997
0.9999999999999997
1.0000000000000007
0.9999999999999993
0.9999999999999998
1.0000000000000018


La restricción de unicidad para el segundo solver es:

In [31]:

u2 = np.zeros(7)
for i in range(7):
  u2[i] = sum(res_sol3[i])
  print(sum(res_sol3[i]))

1.0000000000000002
1.0000000000000024
1.0000000000000002
0.9999999999999999
1.0000000000000024
0.9999999999999987
1.0000000000000047


La solución a la función objetivo para el primer solver es:

In [32]:
s1 = np.zeros(7)
for i in range(7):
  s1[i] = res_sol1[i]@S@res_sol1[i]
  print(res_sol1[i]@S@res_sol1[i])

9.397644610398304e-05
0.00012574783889287994
0.00016502479794085683
0.0002118073232479138
0.00026609541481405164
0.0003278890726392688
0.0003971882967235673


La solución a la funcióón objetivo para el segundo solver es:

In [33]:
s2 = np.zeros(7)
for i in range(7):
  s2[i] = res_sol2[i]@S@res_sol3[i]
  print(res_sol2[i]@S@res_sol3[i])

9.397644610366364e-05
0.00012574783889287943
0.00016502479794085575
0.00021180732324791287
0.0002660954148140495
0.00032788907263593144
0.0003971882967235453


In [ ]:
tabla = pd.DataFrame({'||Dif||': norm2,
                     '|Dif|': norm1,
                     'w1*mu': mu1,
                     'w2*mu': mu2,
                     'Sigma1': s1,
                     'Sigma2': s2})

In [35]:
tabla

,||Dif||,|Dif|,w1*mu,w2*mu,Sigma1,Sigma2
0,1.411129e-11,6.573456e-11,0.4,0.4,0.000094,0.000094
1,1.622015e-11,7.967338e-11,0.5,0.5,0.000126,0.000126
2,1.578511e-11,6.933233e-11,0.6,0.6,0.000165,0.000165
3,2.458990e-11,1.033076e-10,0.7,0.7,0.000212,0.000212
4,2.322346e-11,9.366459e-11,0.8,0.8,0.000266,0.000266
5,2.237532e-11,9.864392e-11,0.9,0.9,0.000328,0.000328
6,2.747350e-11,1.199407e-10,1.0,1.0,0.000397,0.000397



**Soluciones de método de Lagrange ($L$) vs Newton usando diferencias finitas ($N_{df}$)**

Ahora mostramos el mismo comparativo, pero ahora entre el modelo con multiplicadores de Lagrange y el algoritmo de Newton usando las expresiones de las derivadas de a través de funciones simbólicas (véase el código de la implementación para mayor detalle):

In [36]:
print(tabla.to_markdown())

|    |     ||Dif|| |       |Dif| |   w1*mu |   w2*mu |      Sigma1 |      Sigma2 |
|---:|------------:|------------:|--------:|--------:|------------:|------------:|
|  0 | 1.41113e-11 | 6.57346e-11 |     0.4 |     0.4 | 9.39764e-05 | 9.39764e-05 |
|  1 | 1.62202e-11 | 7.96734e-11 |     0.5 |     0.5 | 0.000125748 | 0.000125748 |
|  2 | 1.57851e-11 | 6.93323e-11 |     0.6 |     0.6 | 0.000165025 | 0.000165025 |
|  3 | 2.45899e-11 | 1.03308e-10 |     0.7 |     0.7 | 0.000211807 | 0.000211807 |
|  4 | 2.32235e-11 | 9.36646e-11 |     0.8 |     0.8 | 0.000266095 | 0.000266095 |
|  5 | 2.23753e-11 | 9.86439e-11 |     0.9 |     0.9 | 0.000327889 | 0.000327889 |
|  6 | 2.74735e-11 | 1.19941e-10 |     1   |     1   | 0.000397188 | 0.000397188 |




Por lo anterior podemos concluir que en las hipótesis de este segundo comparativo:

* La diferencia entre las soluciones aumenta conforme aumenta el rendimiento deseado, pero en una magnitud mucho menor que con diferencias finitas.

* Ambas soluciones sigen cumpliendo con las restricciones deseadas.

* El riesgo con el solver usando los multiplicadores de Lagrange es prácticamente el mismo que el del solver con el método de Newton. 

* Aparenemtemente, las soluciones estimadas por ambos solvers son muy cercanas.


Además, de tales comparativos se puede afirmar que el riesgo con el solver usando los multiplicadores de Lagrange siempre es igual o menor que el del solver con el método de Newton utilizando diferencias finitas y es igual con diferencias simbólicas.
